# Lesson 13: Updating User Profiles 
---
Intro: We will continue working on our user profile page allowing users to update their own profiles without going to the admin page. 

# Review
---

1. What is a one-to-one relationship regarding users and their profiles?
2. What is Pillow?
3. What command do you use to open the Django shell?
4. What is a Django shell?

# Concept 1: Django Signals
---


## Objective
Finally, we will create a receiver function for a Django signal that will make sure our profiles are created when a user first registers. So whenever a new user registers to our site, they also will make a profile.
## What are they? 

Signals allow certain senders to notify a set of receivers that some action has taken place. They’re especially useful when many pieces of code may be interested in the same events. 

For example, we don't want to sit outside all day and wait for the postal service to drop off mail. The mailman can signal there is mail by raising the mailbox flag. If the flag is down, then there is no mail. 

1. In the users directory, create a file called `signals.py`. This is outside of the templates folder because we want other models to access these signals.
44. Import the following - 
```
from django.db.models.signals import post_save
from django.contrib.auth.models import User
from django.dispatch import receiver
from .models import Profile
```
* `post_save` - This is a signal that gets fired after an object is saved. We need this when a user is created.
* `User` - This will be the "sender" that sends the signal.
* `receiver` - A receiver receives the signal and performs some tasks.
* `Profile` - We need this Profile model since we will be creating a model. 

So remember our objective, we need a signaling mechanism to signal a new user registration that creates a new profile.
3. `def create_profile(sender, instance, created, **kwargs):` - This function creates a profile for each new user.
46. Inside this function, include the following - 
```
 if created:
        Profile.objects.create(user=instance)
```
* This simply states if a user is created, create a profile under this user instance.
4. Include this decorator on top of the create_profile function - 
```
@receiver(post_save, sender=User)
```
* It takes the signal and sends it to the sender or the User. When a user is saved, then send this signal. Once the receiver receives that signal, it goes through the function to create a profile given a created user. Overall, just know that a signal is needed from the user creation part in order to create a profile.
5. Now we need a function that saves the profile given the same signal direction - 
```
@receiver(post_save, sender=User)
def save_profile(sender, instance, **kwargs):
    instance.profile.save()
```
* The decorator is similar where it needs a signal from the User to create a profile.
* The instance saves the profile created.
6. Now we need to import our signals inside a ready function to explicitly tell Django that we are ready for signals to be received and sent. So go to `apps.py` under the users directory.
50. Inside the UsersConfig class, created this new method - 
```
def ready(self):
        import users.signals
```
* As it's stated, the ready method is called once Django is ready to receive or send signals then it imports them.
8. Now let's go to the browser, register a new user, and check out their profile.

## DIY:
---

1. What are signals?
3. What are the components of a signal?
2. Give a real life example for signals in action?

# Concept 2: User Profile Forms
---


## Objective
In order to update our user and profile, we need to create forms. The fields on the forms allow users to edit their profiles. This will be very similar to editing our posts.
## Form Creation
9. Within the users app, open the `forms.py` file. We're going to create a Model Form which allows us to work with a specific database model. In this case, we want a form that will update our user model.
10. `class UserUpdateForm(forms.ModelForm):` - In this class, we inherit from Django's model form where we will edit the username and email fields. This will be very similar to the previous function so copy everything inside the `UserRegisterForm` function and paste it into `UserUpdateForm`. Then delete `password1, password2`. In the future, we will have a function to reset the password.
* For review, `email = forms.EmailField()` - This provides an email field where users input their email address.
* `class Meta:` - The importance of a class is to encapsulate data. This Meta class encapsulates the User model and fields. If we didn't have this class, then we would have to explicitly add each field of the user model.
11. `from .models import Profile` - Next, we need to import the Profile model because we need access to the image field.
12. `class ProfileUpdateForm(forms.ModelForm):` - Create another function called ProfileUpdateForm that inherits from the ModelForm. 
13. Now inside the `ProfileUpdateForm` function, create a Meta class (similar to the previous functions). Set `model = Profile` and ` fields = ['image']`.
14. Overall `forms.py` should have these additions:

In [ ]:
from .models import Profile

...

# Updates username and email
class UserUpdateForm(forms.ModelForm):
    email = forms.EmailField()

    class Meta:
        model = User
        fields = ['username', 'email']

# Updates image
class ProfileUpdateForm(forms.ModelForm):
    class Meta:
        model = Profile
        fields = ['image']

15. Now open the `views.py` file in the users directory. Remember that the view renders our page onto the screen. 
16. We need to import the previously defined forms so at the top where we import from `.forms` include `UserUpdateForm, ProfileUpdateForm`
17. In the profile method, let's create instances of these. We'll name them u_form for the user form and p_form for the profile form.
```
def profile(request):
      u_form = UserUpdateForm()
      p_form = ProfileUpdateForm()
```
18. Below the instances, create a context dictionary. Recall that the context dictionary provides key and value pairs as context in which the template is being rendered -
```
context = {
        'u_form': u_form,
        'p_form': p_form
    }
```
19. Next, we need to return the context so in the render function, as a third argument, include context -
`return render(request, 'users/profile.html', context)`

## DIY:
---

1. What is the role of a context dictionary?
2. Why do we need `class Meta`?
3. What are the differences and similarities between the UserUpdateForm and the ProfileUpdateForm?

# Concept 3: Pre-Populate Data
---


## Objective
Now let's go to the profile template to display these forms. We did this already when creating a register page so we will use that for inspiration.
## What are they?
20. Within the user's templates, open `register.html` and copy the entire form from the opening to the closing tag.
21. Go to `profile.html` and paste the code where it says FORM HERE.
22. Recall that we have 2 forms (user and profile forms). We're going to put the fields for both of these forms into a single HTML form so that the user sees it as one.
23. Edit `{{ form | crispy }}` so it becomes `{{ u_form | crispy }}`. Below that create another form for the profile - `{{ p_form | crispy }}`
24. Let's make changes to the form. Edit the legend to replace `Join Today` with `Profile Info`. Replace `Sign Up` with `Update`.
25. Inside the opening form tag, we need to include an encode type. To encode means to convert into a coded form. Edit the form tag so it looks like this - 
```
<form method="POST" enctype="multipart/form-data">
```
* This value is required when you are using forms that have a file upload control. We are uploading images to the server.
26. Let's open up the browser and reload the profile page. 
27. We want our information to be pre-populated so we can view our current username and email. So let's make this change.
28. Go to the profile function in the `views.py` from the users directory. To do this, we need to save the current instance of these forms by passing the instance through the forms -
``` 
u_form = UserUpdateForm(instance=request.user)
p_form = ProfileUpdateForm(instance=request.user.profile)
``` 
* This small change will populate those forms with the current users' information.
29. Now we need to check if this is a POST route or not because we need to post this data onto the server. If it is a POST route, then we'll want to see if our forms are valid. If they are valid, save the information. Right above the forms include this if statement - 
```
if request.method == 'POST':
```
30. Indent the forms so they become under the if statement.
31. Now place an `else:` below them. Copy the `u_form` and `p_form` variables and paste them inside the else section.
32. Now we want to pass in the POST data into our forms. Edit the arguments to look like this - 
```
u_form = UserUpdateForm(request.POST, instance=request.user)
p_form = ProfileUpdateForm(request.POST, request.FILES, instance=request.user.profile)
```
* Within the arguments and as the first argument of both forms, include this `request.POST`. This ensures these forms are requesting a POST request.
* Also for the profile form, we will also be getting some additional image data along with the POST request. Do this by adding the argument to the p_form - `request.FILES`. 
33. Now we have to check if both forms are valid. If they are invalid, then don't save any data. Within the first if statement and below the form variables include these. Please type this out - 
```
if u_form.is_valid() and p_form.is_valid():
      u_form.save()
      p_form.save()
      messages.success(request, f'Your account has been updated!')
      return redirect('profile')  
```  
* The if statement checks if both forms are valid.
* If they are, save them and present a message. Finally, redirect them to the profile.
34. Let's try it out in the browser. Reload the profile page and change the username.
35. Check the admin page to ensure no duplicate users were created. 

In [ ]:
@login_required
def profile(request):
    if request.method == 'POST':
        u_form = UserUpdateForm(request.POST, instance=request.user)
        p_form = ProfileUpdateForm(request.POST,
                                   request.FILES,
                                   instance=request.user.profile)
        if u_form.is_valid() and p_form.is_valid():
            u_form.save()
            p_form.save()
            messages.success(request, f'Your account has been updated!')
            return redirect('profile')

    else:
        u_form = UserUpdateForm(instance=request.user)
        p_form = ProfileUpdateForm(instance=request.user.profile)

    context = {
        'u_form': u_form,
        'p_form': p_form
    }

    return render(request, 'users/profile.html', context)

## DIY:
---

1. What is a POST request?
2. What changes were made to pre-populated our information?
3. What does encode mean?

# Concept 4: Profile Pictures on the Home Page
---


## Objective
This is a short concept where we will display all profile pictures on the home page. 
## What are they?
36. Open the `home.html` page from `blog/templates/blog`. Recall that this is where our code for the home page lies.
37. In between the opening article tag and the first div tag, place this code in -
```
<img class="rounded-circle article-img" src="{{ post.author.profile.image.url }}">
```
* We provide an image tag with a rounded circle class. We also provide the source url of the image.
38. Lastly, open the browser to view the changes.
39. That's it! Push your code to GitHub.

## DIY:
---

1. In the code block below, create an image tag with a class of rounded-circle that has a source url of `../assets/images/dog.png`

# Summary:
---


1. Give an example of signals in real life and in Django's use.
2. What is the purpose of the Meta class?
3. What does encode mean?
4. How are context dictionaries used when rendering a template?

# Homework:
---



## Objective
Organize/ structure code & User Input

1. If you're finished with blackjack, now combine the war game and blackjack. Let the user decide which game to play. This should be straightforward. Encapsulate your two games into different functions and then create if statements to decide what game the user wants to play.

# Notes on homework:
---

I will check in on Thursday,  through email to check on your progress. Respond with any questions you might have. Otherwise, a simple “all good” is appropriate if you have no questions or comments. 

You will need to upload your coding homework assignments to GitHub.
1. In gitbash, change directories to the homework directory: tomas_python/homework
* TIP: use ‘cd’ to change directories
* Use ‘cd ..’ to return to the previous directory
* Use ‘pwd’ to show full pathname of the current working directory 
* Use ‘ls’ to list all your directories
2. Once you’re in that directory, type in ‘git pull’
* This ensures you have all updated files
* If there is an error involved, email me immediately so we can try resolving it.
* Otherwise, type your code below and we’ll resolve issues next class
3. To create a new file, type in ‘touch hw01.py’ or the appropriate file name
* ‘Touch’ creates a new file
4. Open up the python file and start coding!

Note: Become familiar with these actions. This is essentially what happens in the backend when you right-click and create a new folder/file!


# Update
---


Error can be different from Mac/Windows users.
If this error occurs: 
`save() got an unexpected keyword argument 'force_insert'` then update the save method in users/models.py to the following:
```
def save(self, *args, **kwargs):
    super().save(*args, **kwargs)
```